<a href="https://colab.research.google.com/github/robsonbfreitas/CalculadoraDeAluguel/blob/main/AnaliseExploratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.vivareal.com.br/venda/sp/sao-paulo/apartamento_residencial/'

In [ ]:
c = requests.get(url).content

In [ ]:
soup = BeautifulSoup(c)

In [ ]:
aluguel_tag = soup.find_all('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'})

In [ ]:
len(aluguel_tag)
paginas = ['']
for i in range(2,40):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)


In [ ]:
import pandas as pd

In [ ]:
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,vaga_lista,area_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista,
            'vaga':vaga_lista,
            'area':area_lista}
    df = pd.DataFrame(data)
    return df


In [ ]:
URL = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/{}'

In [ ]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)



In [ ]:
df_final = pd.concat(df_lista)

In [ ]:
df_final.head()

In [ ]:
df_final['bairro'] = df_final.endereco.str.split('\s-').str[1]
df_final['rua'] = df_final.endereco.str.split('\s-').str[0]
df_final['bairro'] = df_final.bairro.str.split(',').str[0]

In [ ]:
df_final.head()

In [ ]:
df_final['quartos'] = df_final.quartos.str.extract('(.*) Quarto')
df_final['banheiro'] = df_final.banheiro.str.extract('(.*) Banheiro')
df_final['vaga'] = df_final.vaga.str.extract('(.*) Vaga')
df_final['area'] = df_final.area.str.extract('(.*)  m²')


In [ ]:
df_final.head()

In [ ]:

 df_final['aluguel'] = df_final.aluguel.str.extract('R\$ (.*) /Mês')
 df_final['aluguel'] = df_final['aluguel'].replace('\.', '', regex=True)

In [ ]:
df_final.head()

In [ ]:
df_teste = df_final

In [ ]:
colunas = df_teste.drop(columns=['endereco','bairro','rua'])

In [ ]:
colunas

In [ ]:
df_teste.head()